In [1]:
%env KERAS_BACKEND=theano
import MSRII
from c3d import C3DFeatureNet
from video import Video

env: KERAS_BACKEND=theano


Using Theano backend.


In [3]:
%reload_ext autoreload 
%autoreload 1
%aimport c3d
%aimport video

c3dnet = C3DFeatureNet(feature_file=None)
c3dnet.load()

C3D-Network is ready.


In [7]:
video = Video('/data-disk/MSRII/videos/1.avi')

y = c3dnet.extract_feature(video)

In [9]:
def FC4Net():
    from keras.models import Sequential
    from keras.layers.core import Dense, Dropout
    model = Sequential()
    model.add(Dense(128, input_dim=4096, init='uniform', activation='sigmoid'))
    model.add(Dropout(0.35))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.45))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.55))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
    return model

fc4net = FC4Net()
fc4net.load_weights('../network/FC4Net_weights.h5')

In [14]:
pred = fc4net.predict(y, batch_size=32)
dataset = MSRII.Dataset('/data-disk/MSRII/')

In [93]:
import numpy as np


seg_metas = dataset.video_metas[0].seg_metas
target = np.zeros(len(video.frames))
for seg_meta in seg_metas:
    s = seg_meta['start']
    t = seg_meta['duration']
    target[s:s + t] = 1

predict = np.zeros(len(video.frames))
p = pred.reshape(len(pred))
for t in np.nditer(np.where(p == 1)):
    predict[t * 16:(t + 1) * 16] = 1

交集 = np.logical_and(target == 1, predict == 1)
聯集 = np.logical_or(target == 1, predict == 1)

sum(交集 == 1) / sum(聯集 == 1)

0.63537906137184119